In [1]:
import os
import datetime
import pandas as pd
import re
import great_expectations
import agriculture as ag

In [2]:
#!pip install great_expectations

## Descargando los datos

Para descargar los datos referentes a los precios de diversos productos agricolas, podemos ejecutar directamente el script `agriculture.py` en la terminal de la siguiente manera:

```python3 agriculture.py```

El script mencionado llama de forma iterativa a la API de la pagina del Sistema Nacional de Informacion de Mercados (SNIIM) de la Secretaria de Economia, para recuperar los precios de diversos productos en diversos centros de distribucion en el pais, desde el año 2020 hasta la fecha.

Otra manera de descargar los datos es ejecutando la siguiente celda, la cual inicializa un objeto de tipo `ScrapperMarketAgriculture`, el cual contiene la funcion `scraping`, la cual realiza el mismo proceso del script mencionado anteriormente.

In [3]:
# ESTE SCRAPPER PUEDE TARDAR, RECOMENDABLE EJECUTAR DIRECTAMENTE DESDE TERMINAL
scrapper = ag.ScrapperMarketAgriculture() # Generamos un objeto de tipo ScrapperMarketAgriculture (scrapper SNIIM)
scrapper.scraping() # Scrapear la pagina del SNIIM para las frutas y hortalizas

Producto: Acelga - Primera
Producto: Aguacate Criollo - Primera
Producto: Aguacate Fuerte - Primera
Producto: Aguacate Hass - Calidad extra
Producto: Aguacate Hass - Calidad super extra
Producto: Aguacate Hass - Primera
Producto: Aguacate Hass adelantado - Primera
Producto: Aguacate Hass flor vieja - Primera


KeyboardInterrupt: 

Una vez recuperados los datos, estos se almacenan de forma local en archivos .csv, los cuales se encuentran en el directorio `data/sniim`

In [5]:
# Directorio que contiene los archivos generados por el scrapper
sniim_dir = os.path.join("..","..","data","sniim")

## Procesamiento de los datos

Como al final tendremos una dataframe conteniendo los precios de diversos productos, podemos tratar de obtener el nombre de cada producto a partir del nombre del archivo csv, para esto definimos la siguiente funcion:

In [7]:
# Definimos una funcion para obtener el nombre del cultivo a partir del nombre del archivo csv
def estandarizar_nombre(nombre : str) -> str:
    """
    Regresa el nombre del cultivo/producto a partir del nombre del archivo .csv
    """
    cultivo_std = nombre.split('_')[:-1]
    cultivo_std = '_'.join(cultivo_std)
    cultivo_std = re.sub(r"(?<=#)_(?=\d)",'',cultivo_std)
    cultivo_std = re.sub("_"," ",cultivo_std)
    cultivo_std = re.sub("csemilla", "con semilla", cultivo_std)
    cultivo_std = re.sub("ssemilla", "sin semilla", cultivo_std)
    cultivo_std = cultivo_std.title()

    return cultivo_std

Finalmente, podemos iterar sobre los archivos presentes en el directorio anterior, para generar una dataframe por cada uno de ellos, agregar las dataframes a una lista para posteriormente concatenarlas en una sola.

Ademas, a cada dataframe le agregamos la columna `producto` para poder diferenciar las entradas una vez concatenadas las dataframes, y separamos la columna `destino` en dos: `destino` y `central` para identificar mas facilmente la Entidad de destino y la central de abastos donde se levanto la muestra.

In [8]:
# Almacenaremos nuestras dataframes en una lista para despues concatenar
df_list = []
for dirpath, subdir, files in os.walk(sniim_dir):
    for file in files:
        if file == ".DS_Store":
            continue
        
        df = pd.read_csv(os.path.join(dirpath,file)) 
        df["producto"] = estandarizar_nombre(file) # Obtenemos el nombre del producto a partir del nombre del archivo
        df[["destino","central"]] = df["destino"].str.split(": ", expand=True) # Separamos la columna destino en 2
        
        # Reordenamos las columnas
        cols = df.columns.to_list()
        del cols[-1]
        cols.insert(4,"central")
        
        del cols[-1]
        cols.insert(1,"producto")
        
        df = df[cols]
        
        # Agregamos la dataframe generada a la lista
        df_list.append(df)

In [9]:
precios_sniim = pd.concat(df_list,ignore_index=True) # Concatenamos nuestra lista de dataframes en una sola

Una vez obtenida nuestra dataframe final, podemos utilizar el metodo `sample` para obtener una vista general de nuestra dataframe.

In [10]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec,obs
1373881,13/07/2022,Nectarina,Caja de 10 kg.,Importación,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",50.0,60.0,55.00,NaN
886863,04/11/2022,Espinaca,Manojo,Baja California,Baja California,"Central de Abasto INDIA, Tijuana",15.0,15.0,15.00,NaN
1486717,11/07/2023,Papa Gema,Caja de 24 kg.,Veracruz,Veracruz,Central de Abasto de Jalapa,29.17,29.58,29.58,NaN
1152411,24/02/2021,Mandarina,Kilogramo,Nuevo León,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",10.0,13.0,10.00,NaN
986718,20/03/2020,Jicama,Costal de 30 kg.,Distrito Federal,Yucatán,Central de Abasto de Mérida,7.67,7.67,7.67,NaN
1948291,20/05/2022,Uva Superior,Caja de 8 kg.,Importación,Jalisco,Mercado de Abasto de Guadalajara,75.0,81.25,75.00,NaN
751060,16/12/2022,Coliflor Mediana,Pieza,Puebla,Chiapas,Central de Abasto de Tuxtla Gutiérrez,16.0,18.0,16.00,NaN
1742489,08/03/2021,Sandia Sangria,Kilogramo,Jalisco,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",9.0,13.0,10.00,NaN
48422,01/08/2022,Ajo Morado,Caja de 10 kg.,Zacatecas,Guanajuato,Central de Abasto de León,80.0,81.0,80.00,NaN
1813083,07/04/2021,Tomate Saladette,Caja de 28 kg.,Campeche,Yucatán,Central de Abasto de Mérida,5.36,5.36,5.36,NaN


Para una mayor facilidad al momento de comprender nuestra dataframe, generamos un diccionario de datos para esta, explicando lo que representa cada una de las columnas.

In [11]:
precios_dict = {
    "fecha": "Fecha en la que se levanto la encuesta",
    "product": "Producto al que hace referencia la entrada/muestra",
    "presentacion": "Presentacion del producto al que se hace referencia",
    "origen": "Entidad Federativa de donde proviene el producto",
    "destino": "Entidad Federativa hacia donde llego el producto",
    "central": "Central de abastos/centro de distribucion de donde se obtuvo la informacion",
    "precio_min": "El valor más bajo de la cotización dentro de una muestra (MXN)",
    "precio_max": "El valor más alto de la cotización dentro de una muestra (MXN)",
    "precio_frec": "Es el dato que más se repite en la muestra (moda) (MXN)",
    "obs": "Observaciones encontradas para la muestra"
}

In [12]:
dict_df = pd.DataFrame.from_dict(precios_dict,orient="index")
dict_df.columns = ["Descripcion"]
dict_df.rename_axis("Columna", inplace=True)
dict_df

,Descripcion
Columna,
fecha,Fecha en la que se levanto la encuesta
product,Producto al que hace referencia la entrada/mue...
presentacion,Presentacion del producto al que se hace refer...
origen,Entidad Federativa de donde proviene el producto
destino,Entidad Federativa hacia donde llego el producto
central,Central de abastos/centro de distribucion de d...
precio_min,El valor más bajo de la cotización dentro de u...
precio_max,El valor más alto de la cotización dentro de u...
precio_frec,Es el dato que más se repite en la muestra (mo...


In [13]:
dict_dir = os.path.join("..","..","diccionarios") 
if not os.path.exists(dict_dir):
    os.mkdir(dict_dir)
    
dict_df.to_csv(os.path.join(dict_dir,"diccionario_sniim.csv"),encoding="iso-8859-1")

## Limpieza de los datos

Una vez generado nuestro diccionario, podemos comenzar a tratar los datos de nuestro dataframe. Algo que podemos observar del sample de nuestro dataframe es que la columna `obs` parece ser la unica que cuenta con valores nulos. Podemos tener una idea de la proporcion de datos faltantes en esta columna ejecutando la siguiente celda, la cual calcula el porcentaje de entradas faltantes en la misma.

In [14]:
print(f"Porcentaje de entradas faltantes en columna 'obs': {100* precios_sniim['obs'].isna().sum() / precios_sniim.shape[0]:.2f}%")

Porcentaje de entradas faltantes en columna 'obs': 96.03%


Vemos que la mayoria de las entradas en nuestra dataframe (> 90%) no cuenta con informacion en la columna `obs`, por lo que puede ser mas conveniente deshacernos de esta columna.

In [15]:
precios_sniim.drop(columns="obs", inplace=True)

In [16]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
1271520,27/09/2022,Manzana Red Delicious,Caja de 20 kg.,Chihuahua,Veracruz,Central de Abasto de Jalapa,32.0,32.5,32.0
693112,13/06/2023,Cilantro,Caja de 5 kg.,Puebla,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",50.0,60.0,56.0
758395,17/09/2021,Coliflor Sin Clasificacion,Pieza,Distrito Federal,Yucatán,Central de Abasto de Mérida,30.0,32.0,32.0
1975034,07/02/2023,Zanahoria Leña,Arpilla de 20 kg.,Zacatecas,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",7.5,10.0,9.0
110444,02/10/2023,Betabel,Kilogramo,Hidalgo,Hidalgo,Central de Abasto de Pachuca,60.0,65.0,65.0
891423,11/11/2022,Espinaca,Manojo,Hidalgo,Querétaro,Mercado de Abasto de Querétaro,55.0,60.0,60.0
1607828,21/06/2023,Piña Mediana,Pieza,Veracruz,Chiapas,Central de Abasto de Tuxtla Gutiérrez,30.0,35.0,30.0
619209,27/01/2023,Chile Poblano,Kilogramo,Distrito Federal,Campeche,"Mercado ""Pedro Sáinz de Baranda"", Campeche",40.0,45.0,45.0
904150,09/01/2020,Fresa,Caja de 5 kg.,Nuevo León,Tamaulipas,Módulo de Abasto de Reynosa,60.0,62.0,60.0
1745462,07/12/2022,Sandia Sangria,Kilogramo,Coahuila,Coahuila,"Central de Abasto de La Laguna, Torreón",10.5,11.0,11.0


Posteriormente, podemos utilizar el metodo `info` de nuestra dataframe para obtener informacion acerca de los tipos de datos de las columnas o la posible presencia de mas datos faltantes.

In [17]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   fecha         object 
 1   producto      object 
 2   presentacion  object 
 3   origen        object 
 4   destino       object 
 5   central       object 
 6   precio_min    object 
 7   precio_max    object 
 8   precio_frec   float64
dtypes: float64(1), object(8)
memory usage: 138.7+ MB


Lo primero que notamos es que la columna `fecha` tiene un tipo de dato `object`, lo cual puede significar que esta almacenada como un string. Al tratarse de datos temporales, podemos convertir esta columna a un objeto de tipo `datetime` para una mayor facilidad al momento de manipular o explorar los datos.

In [21]:
precios_sniim["fecha"] = pd.to_datetime(precios_sniim["fecha"], format="%d/%m/%Y")

Despues podemos analizar los valores en la columna `origen` para verificar que no haya valores inesperados. Los posibles valores en esta columna son los nombres de los 32 estados de Mexico, o bien "Importacion" si el producto fue importado del extranjero, o "Nacional" si no hay datos sobre su estado de origen.

In [22]:
precios_sniim["origen"].unique()

array(['Puebla', 'Baja California', 'San Luis Potosí', 'Chiapas',
       'Durango', 'Guanajuato', 'Aguascalientes', 'Michoacán', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Jalisco', 'Distrito Federal',
       'Quintana Roo', 'Hidalgo', 'México', 'Yucatán', 'Tamaulipas',
       'Veracruz', 'Zacatecas', 'Importación', 'Sonora', 'Guerrero',
       'Sinaloa', 'Morelos', 'Nacional', 'Sin Especificar', 'Chihuahua',
       'Campeche', 'Coahuila', 'Tabasco', 'Baja California Sur',
       'Querétaro', 'Colima'], dtype=object)

Vemos que no existen anomalias aparentes en esta columna, asi que podemos proceder a revisar los valores de la columna `destino`, la cual deberia tener valores similares a la columna anterior.

In [23]:
precios_sniim["destino"].unique()

array(['Baja California ', 'Coahuila', 'Chiapas', 'DF', 'Durango',
       'Guanajuato', 'Michoacán', 'Morelos', 'Nayarit', 'Nuevo León',
       'Oaxaca', 'Quintana Roo', 'San Luis Potosí', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Veracruz', 'Sinaloa', 'Chihuahua', 'Guerrero',
       'Campeche', 'Yucatán', 'Aguascalientes', 'Baja California Sur',
       'Colima', 'Jalisco', 'Hidalgo', 'México', 'Puebla', 'Querétaro',
       'Zacatecas'], dtype=object)

Vemos que tampoco hay anomalias en esta columna. Sin embargo, algunos valores contienen espacios al final del valor (por ejemplo, `Baja California `). Esto se puede corregir facilmente utilizando la funcion `strip` que nos ofrece pandas:

In [24]:
precios_sniim["destino"] = precios_sniim["destino"].str.strip()
precios_sniim["destino"].unique()

array(['Baja California', 'Coahuila', 'Chiapas', 'DF', 'Durango',
       'Guanajuato', 'Michoacán', 'Morelos', 'Nayarit', 'Nuevo León',
       'Oaxaca', 'Quintana Roo', 'San Luis Potosí', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Veracruz', 'Sinaloa', 'Chihuahua', 'Guerrero',
       'Campeche', 'Yucatán', 'Aguascalientes', 'Baja California Sur',
       'Colima', 'Jalisco', 'Hidalgo', 'México', 'Puebla', 'Querétaro',
       'Zacatecas'], dtype=object)

Finalmente, la columna `precio_max` aparece como de tipo `object`, por lo que posiblemente hay valores que impidieron que fuera parseada a `float`.

Despues de una inspeccion de esta columna, vemos que para algunos precios, se utiliza la coma (,) para separar los millares. Podemos eliminar esta coma de nuestro dataframe con el metodo `replace`, para despues transformarla a tipo `float` con el metodo `astype(float)`.

In [25]:
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(str).str.replace(",","")
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(float)

In [26]:
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(str).str.replace(",","")
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(float)

Finalmente, podemos volver a obtener una muestra de nuestro dataframe para ver su estructura final, asi como volver a llamar al metodo `info` para verificar los tipos de datos de cada columna y que no haya valores faltantes.

In [27]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
1993753,2022-11-16,Zanahoria Mediana,Arpilla de 25 kg.,Puebla,DF,Central de Abasto de Iztapalapa DF,3.6,4.40,4.00
1635528,2021-04-23,Platano Dominico,Kilogramo,Veracruz,Jalisco,Mercado de Abasto de Guadalajara,14.5,15.00,14.50
1068953,2023-01-04,Lechuga Romanita Mediana,Docena,Puebla,Morelos,Central de Abasto de Cuautla,12.5,14.17,13.33
1724631,2022-03-22,Sandia Negra,Kilogramo,Coahuila,Chihuahua,Central de Abasto de Chihuahua,7.0,11.00,10.00
1000179,2022-05-23,Jicama,Kilogramo,Nayarit,Nayarit,Nayarabastos de Tepic,15.0,17.00,16.00
908490,2021-10-13,Fresa,Caja de 5 kg.,Importación,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",60.0,66.00,66.00
592472,2023-08-24,Chile Pimiento Morron,Kilogramo,Veracruz,Veracruz,Central de Abasto de Minatitlán,52.0,57.00,52.00
411847,2022-04-18,Chile Cat,Caja de 10 kg.,Yucatán,Yucatán,"Mercado ""Casa del Pueblo""",50.0,50.00,50.00
669061,2020-11-25,Cilantro,Caja de 5 kg.,Nuevo León,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",22.0,24.00,24.00
1948289,2022-05-18,Uva Superior,Caja de 8 kg.,Importación,Jalisco,Mercado de Abasto de Guadalajara,75.0,81.25,75.00


In [28]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype         
---  ------        -----         
 0   fecha         datetime64[ns]
 1   producto      object        
 2   presentacion  object        
 3   origen        object        
 4   destino       object        
 5   central       object        
 6   precio_min    float64       
 7   precio_max    float64       
 8   precio_frec   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 138.7+ MB


Finalmente, almacenamos nuestro dataframe tidy en un archivo `.parquet`.

In [29]:
clean_data_dir = os.path.join("..","..","data","clean_data")

if not os.path.exists(clean_data_dir):
    os.mkdir(clean_data_dir)
    
precios_sniim.to_parquet(os.path.join(clean_data_dir,"precios_sniim.parquet"))

: 